In [22]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [23]:
df = pd.read_csv('/content/main_table_2012_2023 - new_main_table_2012_2023.csv')

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468 entries, 0 to 467
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ville               468 non-null    object
 1   code_commune        468 non-null    object
 2   departement         468 non-null    object
 3   region              468 non-null    object
 4   sport               468 non-null    object
 5   club                468 non-null    object
 6   fin_saison          468 non-null    int64 
 7   division            468 non-null    int64 
 8   classement          468 non-null    int64 
 9   score_classement    468 non-null    int64 
 10  capacite            468 non-null    int64 
 11  taux_remplissage    468 non-null    object
 12  score_event         468 non-null    object
 13  nb_crea_entreprise  468 non-null    object
 14  taux_chomage        390 non-null    object
 15  salaire_median      390 non-null    object
dtypes: int64(5), object(11)
me

Cleaning en float

In [25]:
df['taux_remplissage'] = df['taux_remplissage'].str.replace(',', '.').astype(float)

In [26]:
df['score_event'] = df['score_event'].str.replace(',', '.').astype(float)

In [27]:
df['taux_chomage'] = df['taux_chomage'].str.replace(',', '.').astype(float)

In [28]:
df['nb_crea_entreprise'] = df['nb_crea_entreprise'].str.replace('\u202f', '').str.replace(',', '').astype(int)

In [29]:
df['salaire_median'] = df['salaire_median'].str.replace('\u202f', '').str.replace(',', '.').astype(float)

Création score sportif

In [30]:
df['class'] = (20 - df['classement'] + 1) / 20

In [31]:
df['coeff_division'] = df['division'].map({1: 1, 2: 0.5})

In [53]:
df['score_sportif'] = (0.4*df['class']) + (0.3*df['taux_remplissage']) + (0.4*df['coeff_division']) + (0.1*df['score_event'])

Création score économique

In [33]:
df['salaire_median_norm'] = (df['salaire_median'] - df['salaire_median'].min()) / (df['salaire_median'].max() - df['salaire_median'].min())

In [34]:
df['taux_chomage_norm'] = 1 - (df['taux_chomage'] - df['taux_chomage'].min()) / (df['taux_chomage'].max() - df['taux_chomage'].min())

In [35]:
df['nb_crea_entreprise_norm'] = (df['nb_crea_entreprise'] - df['nb_crea_entreprise'].min()) / (df['nb_crea_entreprise'].max() - df['nb_crea_entreprise'].min())

In [36]:
df['score_economique'] = (0.4 * df['salaire_median_norm']) + (0.5 * df['taux_chomage_norm']) + (0.5 * df['nb_crea_entreprise_norm'])

Affichage de l'évolution de la moyenne des scores

In [54]:
grouped_df = df.groupby('fin_saison', as_index=False).agg({
    'score_sportif': 'mean',
    'score_economique': 'mean',
    'salaire_median': 'mean',
    'taux_chomage': 'mean'
})

In [55]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=grouped_df['fin_saison'],
        y=grouped_df['score_sportif'],
        mode='lines+markers',
        name='Score Sportif',
        line=dict(color='blue'),
        hovertemplate='Saison: %{x}<br>Score Sportif Moyen: %{y:.2f}<br>Classement Moyen: %{customdata}',
        customdata=df['score_classement']
    )
)

fig.add_trace(
    go.Scatter(
        x=grouped_df['fin_saison'],
        y=grouped_df['score_economique'],
        mode='lines+markers',
        name='Score Économique',
        line=dict(color='green'),
        hovertemplate='Saison: %{x}<br>Score Économique: %{y:.2f}'
    )
)

fig.update_layout(
    title="Évolution Moyenne du Score Sportif",
    xaxis_title="Saison",
    yaxis_title="Score Sportif Moyen",
    hovermode="x unified"
)

fig.show()


Affichage de l'évolution des scores par choix de ville

In [43]:
villes = df['ville'].unique()

fig = go.Figure()

# Ville initiale
ville_initiale = villes[0]
filtered_df = df[df['ville'] == ville_initiale].sort_values('fin_saison')

# Ajouter les traces initiales
fig.add_trace(
    go.Scatter(
        x=filtered_df['fin_saison'],
        y=filtered_df['score_sportif'],
        mode='lines+markers',
        name='Score Sportif',
        line=dict(color='blue'),
        hovertemplate=(
            'Saison: %{x}<br>'
            'Score Sportif: %{y:.2f}<br>'
            'Classement: %{customdata[0]}<br>'
            'Division: %{customdata[1]}'
        ),
        customdata=filtered_df[['classement', 'division']].values
    )
)

fig.add_trace(
    go.Scatter(
        x=filtered_df['fin_saison'],
        y=filtered_df['score_economique'],
        mode='lines+markers',
        name='Score Économique',
        line=dict(color='green'),
        hovertemplate='Saison: %{x}<br>Score Économique: %{y:.2f}'
    )
)

# Boutons pour changer la ville
buttons = []

for ville in villes:
    filtered_df = df[df['ville'] == ville].sort_values('fin_saison')

    # Calculer les limites dynamiques pour l'axe y
    y_min = min(filtered_df['score_sportif'].min(), filtered_df['score_economique'].min())
    y_max = max(filtered_df['score_sportif'].max(), filtered_df['score_economique'].max())

    buttons.append(
        dict(
            label=ville,
            method="update",
            args=[
                {
                    "x": [filtered_df['fin_saison'], filtered_df['fin_saison']],
                    "y": [filtered_df['score_sportif'], filtered_df['score_economique']],
                    "customdata": [filtered_df[['classement', 'division']].values, None]
                },
                {
                    "title": f"Évolution des Scores Sportif et Économique - {ville}",
                    "yaxis.range": [y_min, y_max]  # Mettre à jour la plage de l'axe y
                }
            ]
        )
    )

# Configuration des menus et de la mise en page
fig.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down",
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.15,
            yanchor="top"
        )
    ],
    title=f"Évolution des Scores Sportif et Économique - {ville_initiale}",
    xaxis_title="Saison",
    yaxis_title="Score",
    legend_title="Type de Score",
    hovermode="x unified"
)

# Afficher la figure
fig.show()

In [42]:
# Initialiser la figure
fig = go.Figure()

# Utiliser la première ville dans les données agrégées
ville_initiale = df_aggregated['ville'].unique()[0]
filtered_df = df_aggregated[df_aggregated['ville'] == ville_initiale]

# Ajouter les traces pour le graphique
fig.add_trace(
    go.Scatter(
        x=filtered_df['fin_saison'],
        y=filtered_df['score_sportif'],
        mode='lines+markers',
        name='Score Sportif',
        line=dict(color='blue'),
        hovertemplate=(
            'Saison: %{x}<br>'
            'Score Sportif: %{y:.2f}<br>'
            'Classement: %{customdata[0]}<br>'
            'Division: %{customdata[1]}'
        ),
        customdata=filtered_df[['classement', 'division']].values
    )
)

fig.add_trace(
    go.Scatter(
        x=filtered_df['fin_saison'],
        y=filtered_df['score_economique'],
        mode='lines+markers',
        name='Score Économique',
        line=dict(color='green'),
        hovertemplate='Saison: %{x}<br>Score Économique: %{y:.2f}'
    )
)

# Ajouter les boutons pour les villes
buttons = []
for ville in df_aggregated['ville'].unique():
    filtered_df = df_aggregated[df_aggregated['ville'] == ville]

    # Calculer les limites de y pour les deux courbes (sportif et économique)
    y_min = min(filtered_df['score_sportif'].min(), filtered_df['score_economique'].min())
    y_max = max(filtered_df['score_sportif'].max(), filtered_df['score_economique'].max())

    # Ajouter les boutons avec la mise à jour de la plage de y
    buttons.append(
        dict(
            label=ville,
            method="update",
            args=[
                {
                    "x": [filtered_df['fin_saison'], filtered_df['fin_saison']],
                    "y": [filtered_df['score_sportif'], filtered_df['score_economique']],
                    "customdata": [filtered_df[['classement', 'division']].values, None],
                    "yaxis.range": [y_min, y_max]  # Ajuster la plage de l'axe y
                },
                {
                    "title": f"Évolution des Scores Sportif et Économique - {ville}",
                }
            ]
        )
    )

# Mettre à jour la mise en page
fig.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down",
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.15,
            yanchor="top"
        )
    ],
    title=f"Évolution des Scores Sportif et Économique - {ville_initiale}",
    xaxis_title="Saison",
    yaxis_title="Score",
    legend_title="Type de Score",
    hovermode="x unified"
)

# Afficher la figure
fig.show()

NameError: name 'df_aggregated' is not defined